In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import norm
import matplotlib.pyplot as plt
import multiprocessing as mp
import matplotlib.ticker as ticker


In [ ]:
##~~~~~~OOP Version~~~~~~~
class Option:
    def __init__(self,X0=100,r=0.05,sig=0.2,K=100,T=1,M=2):
        self.X0=X0
        self.r=r
        self.sig=sig
        self.K=K
        self.T=T
        self.M=M
    
    #Virtual functions to be overridden by specific sub-classes
    def payoff(self,N_loop,l): #Depends on option type
        raise NotImplementedError()
    def path(self,N_loop,l): #Depends on underlying SDE
        raise NotImplementedError()
    
    #~~~Common functions to all sub-classes~~~#
    ##Interfaces with mlmc algorithm
    def looper(self,Nl,l,Npl=10**4):
        num_rem=Nl #Initialise remaining samples for while loop
        suml=np.zeros(7)
        while (num_rem>0): #<---Parallelise this while loop
            N_loop=min(Npl,num_rem) #Break up Nl into manageable chunks of size Npl, until last iteration
            num_rem-=N_loop #On final iteration N_loop=num_rem, so num_rem will be=0 and loop terminates
            Pf,Pc=self.payoff(N_loop,l)
            sumPf=np.sum(Pf)
            sumPf2=np.sum(Pf**2)
            if l==0:
                suml+=np.array([sumPf,sumPf2,sumPf,sumPf2,0,0,0])
            else:
                dP_l=Pf-Pc #Payoff difference
                sumPc=np.sum(Pc)
                sumPc2=np.sum(Pc**2)
                fg=np.sum(Pc*Pf)
                suml+=np.array([np.sum(dP_l),np.sum(dP_l**2),sumPf,sumPf2,sumPc,sumPc2,fg])
        return suml
    ##MLMC function
    def mlmc(self,eps,N0=10**3):
        '''
        Runs MLMC method for given option (e.g. European) which returns an array of sums at each level: 
        suml=[d_Pl,d_Pl**2,Pl,Pl**2].
        __Inputs__
        option = option instance (with SDE params and coarseness factor M)
        eps = desired accuracy
        N0 = default number of samples to use when initialising new level
        M=coarseness factor

        __Outputs__
        sums = array of sums of payoff differences between coarse and fine paths at each level and
               sum of payoffs at fine level, each column is a level
        N = final number of samples at each level
        '''
        L=2
        M=self.M

        V=np.zeros(L+1) #Initialise variance vector of each levels' variance
        N=N0*np.ones(L+1) #Initialise num. samples vector of each levels' num. samples
        dN=N0*np.ones(L+1) #Initialise additional samples for this iteration vector for each level
        sums=np.zeros((7,L+1)) #Initialise sums array, each column is a level
        sqrt_h=np.sqrt(M**(np.arange(L+1)))

        while (np.sum(dN)>0): #Loop until no additional samples asked for
            for l in range(L+1): 
                num=dN[l]
                if num>0: #If asked for additional samples...
                    sums[:,l]+=self.looper(int(num),l) #Call function which gives sums


            N+=dN #Increment samples taken counter for each level        
            V=np.maximum((sums[1,:]/N)-(sums[0,:]/N)**2,0) #Calculate variance based on updated samples
            sqrt_V=np.sqrt(V)
            Nl_new=np.ceil((2/eps**2)*np.sum(sqrt_V*sqrt_h)*sqrt_V/sqrt_h) #Estimate optimal number of samples at each level
            dN=np.maximum(0,Nl_new-N) #Number of additional samples
            if sum(dN > 0.01*N) == 0:
                if abs(sums[0,-1])/N[-1]>(M-1)*eps*np.sqrt(0.5):
                    L+=1
                    #Add extra entries for the new level and estimate sums with N0 samples 
                    V=np.concatenate((V,np.zeros(1)), axis=0)
                    N=np.concatenate((N,N0*np.ones(1)),axis=0)
                    dN=np.concatenate((dN,N0*np.ones(1)),axis=0)
                    sqrt_h=np.concatenate((sqrt_h,[np.sqrt(M**L)]),axis=0)
                    sums=np.concatenate((sums,np.zeros((7,1))),axis=1)
                    sums[:,L]+=self.looper(N0,L)

        return sums,N

class Euro_GBM(Option):
    def path(self,N_loop,l):
        M=self.M
        r=self.r
        sig=self.sig
        T=self.T
        X0=self.X0
        Nsteps=M**l
        dt=T/Nsteps
        sqrt_dt=np.sqrt(dt)
        #Initialise fine, coarse asset prices; coarse Brownian increment (BI)
        Xf=X0*np.ones(N_loop)
        Xc=X0*np.ones(N_loop)
        dWc=np.zeros(N_loop)
        for j in range(1,Nsteps+1): #Note that if Nsteps=1 (l=0), j=1 and so coarse path not developed
            dWf=np.random.randn(N_loop)*sqrt_dt
            dWc=dWc+dWf #Keep adding to coarse BI every loop until j is integer multiple of M
            Xf+=r*Xf*dt + sig*Xf*dWf
        if j%M==0: #if j is integer multiple of M...
            Xc+=r*Xc*M*dt + sig*Xc*dWc #...Develop coarse path
            dWc=np.zeros(N_loop) #...Re-initialise coarse BI to 0
        return Xf,Xc
    
    def payoff(self,N_loop,l):
        r=self.r
        T=self.T
        K=self.K
        Xf,Xc=self.path(N_loop,l)
        #Calculate payoffs etc.
        Pf=np.exp(-r*T)*np.maximum(0,Xf-K)
        if l==0:
            return Pf,Xc #Just ignore Pc=Xc
        else:
            Pc=np.exp(-r*T)*np.maximum(0,Xc-K) #Payoff at coarse level
            return Pf,Pc   

In [ ]:
##~~~~~Functional Version~~~~~~~
def mlmc(payoff,eps,N0=10**3,M=2,**kwargs):
    '''
    Runs MLMC method for given mlmc_fn (e.g. mlmc_gbm) which returns an array of sums at each level: 
    suml=[d_Pl,d_Pl**2,Pl,Pl**2].
    __Inputs__
    mlmc_fn = function to call at a given level
    eps = desired accuracy
    N0 = default number of samples to use when initialising new level
    M=coarseness factor
    
    __Outputs__
    sums = array of sums of payoff differences between coarse and fine paths at each level and
           sum of payoffs at fine level, each column is a level
    N = final number of samples at each level
    '''
    L=2

    V=np.zeros(L+1) #Initialise variance vector of each levels' variance
    N=N0*np.ones(L+1) #Initialise num. samples vector of each levels' num. samples
    dN=N0*np.ones(L+1) #Initialise additional samples for this iteration vector for each level
    sums=np.zeros((7,L+1)) #Initialise sums array, each column is a level
    sqrt_h=np.sqrt(M**(np.arange(0,L+1)))
    while (np.sum(dN)>0): #Loop until no additional samples asked for
        for l in range(L+1): 
            num=dN[l]
            if num>0: #If asked for additional samples...
                sums[:,l]+=payoff(int(num),l,M,**kwargs) #Call function which gives sums

        
        N+=dN #Increment samples taken counter for each level        
        V=np.maximum((sums[1,:]/N)-(sums[0,:]/N)**2,0) #Calculate variance based on updated samples
        sqrt_V=np.sqrt(V)
        Nl_new=np.ceil((2/eps**2)*np.sum(sqrt_V*sqrt_h)*sqrt_V/sqrt_h) #Estimate optimal number of samples at each level
        dN=np.maximum(0,Nl_new-N) #Number of additional samples
        if sum(dN > 0.01*N) == 0:
            if abs(sums[0,-1])/N[-1]>(M-1)*eps*np.sqrt(0.5):
                L+=1
                #Add extra entries for the new level and estimate sums with N0 samples 
                V=np.concatenate((V,np.zeros(1)), axis=0)
                N=np.concatenate((N,N0*np.ones(1)),axis=0)
                dN=np.concatenate((dN,N0*np.ones(1)),axis=0)
                sqrt_h=np.concatenate((sqrt_h,[np.sqrt(M**L)]),axis=0)
                sums=np.concatenate((sums,np.zeros((7,1))),axis=1)
                sums[:,L]+=payoff(N0,L,M,**kwargs)
    
    return sums,N
def European(Nl,l,M,Npl=10**4,X0=100,r=0.05,sig=0.2,K=100,T=1):
    '''
    Calculates payoff for European call with GBM for underlying. 
    __Inputs__
    l=fine level
    Nl = desired samples of Yl at this level
    T=time period for SDE to be simulated over
    M=coarseness factor
    Npl=default samples per loop to break up Nl into manageable chunks
    X0,r,sig,K,T = params for SDE
    
    __Outputs__
    suml=[dP_l,dP_l**2,sumPf,sum(Pf**2),sumPc,sum(Pc**2),fg]
    dP_l=sum of payoff differences between coarse and fine paths at this level
    Pf=payoffs at fine level
    Pc=payoffs at coarse level
    fg = sum of Pf*Pc
    '''
    num_rem=Nl #Initialise remaining samples for while loop
    suml=np.zeros(7)    
    Nsteps=M**l #Number of fine steps
    dt=T/Nsteps
    sqrt_dt=np.sqrt(dt)
    while (num_rem>0): #<---Parallelise this while loop
        N_loop=min(Npl,num_rem) #Break up Nl into manageable chunks of size Npl, until last iteration
        num_rem-=N_loop #On final iteration N_loop=num_rem, so num_rem will be=0 and 
        Nsteps=M**l
        dt=T/Nsteps
        sqrt_dt=np.sqrt(dt)
        #Initialise fine, coarse asset prices; coarse Brownian increment (BI)
        Xf=X0*np.ones(N_loop)
        Xc=X0*np.ones(N_loop)
        dWc=np.zeros(N_loop)
        for j in range(1,Nsteps+1): #Note that if Nsteps=1 (l=0), j=1 and so coarse path not developed
            dWf=np.random.randn(N_loop)*sqrt_dt
            dWc=dWc+dWf #Keep adding to coarse BI every loop until j is integer multiple of M
            Xf+=r*Xf*dt + sig*Xf*dWf
        if j%M==0: #if j is integer multiple of M...
            Xc+=r*Xc*M*dt + sig*Xc*dWc #...Develop coarse path
            dWc=np.zeros(N_loop) #...Re-initialise coarse BI to 0
            
        #Calculate payoffs etc.
        Pf=np.exp(-r*T)*np.maximum(0,Xf-K)
        sumPf=np.sum(Pf)
        sumPf2=np.sum(Pf**2)
        if l==0:
            suml+=np.array([sumPf,sumPf2,sumPf,sumPf2,0,0,0])
        else:
            Pc=np.exp(-r*T)*np.maximum(0,Xc-K) #Payoff at coarse level
            dP_l=Pf-Pc #Payoff difference
            sumPc=np.sum(Pc)
            sumPc2=np.sum(Pc**2)
            fg=np.sum(Pc*Pf)
            suml+=np.array([np.sum(dP_l),np.sum(dP_l**2),sumPf,sumPf2,sumPc,sumPc2,fg])
            
    return suml